In [1]:
!pip uninstall mex -y

Found existing installation: mex 0.3
Uninstalling mex-0.3:
  Successfully uninstalled mex-0.3


In [2]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
#np.set_printoptions(threshold=sys.maxsize)
import os
import sys
import pytz
import timeit
import datetime
import numpy as np
import scipy.io as sio

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

## Características de la instancia

Utilizamos una instancia m5.2xlarge que entra en el programa de AWS Educate y que hemos ocupado en prácticas anteriores. Esta instancia tiene las siguientes características:

In [4]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:                        4
CPU MHz:                         3117.771
BogoMIPS:                        5000.00
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        4 MiB
L3 cache:                        33 MiB
NUMA node0 CPU(s):               0-7

In [4]:
mat = sio.loadmat('../data/AGG.mat')      #Start time:  09:52:51 PM  End time:  09:52:54 PM
#mat = sio.loadmat('../data/DEGEN3.mat')   #Start time:  10:27:30 PM  End time:  10:30:58 PM
#mat = sio.loadmat('../data/SHIP04L.mat')  #Start time:  09:21:59 PM  End time:  09:21:59 PM
#mat = sio.loadmat('../data/AGG2.mat')     #Start time:  09:53:47 PM  End time:  09:53:53 PM
#mat = sio.loadmat('../data/QAP8.mat')     #Start time:  10:11:35 PM  End time:  10:11:36 PM
#mat = sio.loadmat('../data/QAP15.mat')    #Start time:  10:15:30 PM  End time:  10:15:50 PM
#mat = sio.loadmat('../data/TRUSS.mat')    #Start time:  10:24:53 PM  End time:  10:24:54 PM

In [5]:
A = mat['A']
b = mat['b']
c = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
A.shape

(488, 615)

# Prueba de paquetería `mex`

In [6]:
A = A.toarray()
#b = np.array(b.T.tolist()[0])
#c = np.array(c.T.tolist()[0])

In [7]:
minim = Minimizer(A, b, c)

In [8]:
minim.add_constraints(lbounds,ubounds)

In [10]:
minim.matrix.shape

(1719, 2335)

In [11]:
print("Start time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))
minim.solve()
print("End time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))

Start time:  09:44:59 PM
End time:  09:47:49 PM


**Valores aproximados**

In [35]:
min_approx = minim.get_min()

In [36]:
coeff_approx = minim.get_coeff()

In [12]:
#coeff_approx

**Valores objetivo**

In [15]:
tableau_obj = minim.matrix

In [16]:
n_restr = tableau_obj.shape[0]-1
n_vars = A.shape[1]

In [17]:
c_min_obj = tableau_obj[-1,0:n_vars]

In [18]:
A_min_obj = tableau_obj[0:n_restr, 0:n_vars]

In [19]:
b_min_obj = tableau_obj[0:n_restr, -1]

In [23]:
print("Start time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))
min_obj = linprog(c_min_obj, A_ub=A_min_obj, b_ub=b_min_obj).fun
print("End time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))

Start time:  09:51:40 PM
End time:  09:54:44 PM


In [26]:
coeff_obj = linprog(c, A_ub=A, b_ub=b).x

**Comprobación**

In [45]:
assert min_obj == approx(min_approx, rel=1e-1), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,3) == approx(coeff_approx, abs=1e+40), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", min_obj)
print("El valor aproximado obtenido con mex es: ", min_approx)
#print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj)
#print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx)

El valor objetivo obtenido con scipy es:  -3.553890952290583e+35
El valor aproximado obtenido con mex es:  -3.602879999999999e+35


## Prueba compilación a C

In [5]:
#mex_c
os.chdir("..")
from src.mex_c import general_c as mex_c_g
from src.mex_c import simplex_networks_c as mex_c_sn
from src.mex_c import problem_definition_c as mex_c_pd
from src.mex_c.minimizer_class_c import Minimizer_c
from src.mex_c.maximizer_class_c import Maximizer_c
os.chdir("notebooks")

In [6]:
minim_c = Minimizer_c(A,b,c)

In [7]:
minim_c.matrix.shape

(489, 1105)

In [8]:
print("Start time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))
minim_c.solve()
print("End time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))

Start time:  11:07:29 PM
End time:  11:07:32 PM


**Valores aproximados**

In [9]:
min_approx_c = minim_c.get_min()
coeff_approx_c = minim_c.get_coeff()

In [21]:
#coeff_approx_c

**Comprobación**

In [13]:
assert min_obj == approx(min_approx_c, abs=1e+5), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,5) == approx(coeff_approx_c, abs=1e+8), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", min_obj)
print("El valor aproximado obtenido con mex es: ", min_approx_c)
#print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj)
#print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx)

El valor objetivo obtenido con scipy es:  -81917191.67681439
El valor aproximado obtenido con mex es:  -81917193.41974491
